In [1]:
import torch
import torch.nn as nn
import numpy as np
import pickle

In [2]:
!git clone https://github.com/czhuang/JSB-Chorales-dataset.git

Cloning into 'JSB-Chorales-dataset'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 46 (delta 4), reused 10 (delta 4), pack-reused 36 (from 1)
Receiving objects: 100% (46/46), 2.78 MiB | 15.73 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [3]:
with open('JSB-Chorales-dataset/jsb-chorales-16th.pkl', 'rb') as p:
    data = pickle.load(p, encoding="latin1")

In [4]:
data.keys()

dict_keys(['test', 'train', 'valid'])

In [14]:
print(np.array(data['train'],dtype="object").shape)
print(np.array(data['test'],dtype="object").shape)
print(np.array(data['valid'],dtype="object").shape)

(229,)
(77,)
(76,)
